# Support Vector Machine with Crafted Features

In [1]:
DATA_PREPARATION_VOTE_METHOD = "max_vote_window" # "max_vote_window" or "sum_and_normalize". Decides how to aggregate the predictions of the overlapping windows
SKIP_TRAIN = False # If True, skips the training phase and only runs evaluation with existing checkpoints
EXISTING_CHECKPOINT_KAGGLE_DATASET_ID = "hsm-models" # set to None if you want to train a new model on Kaggle. Else, set to the Kaggle dataset ID where the existing model checkpoints are stored

In [2]:
import os
import sys
import warnings
import gc
import pathlib

if bool(os.environ.get("KAGGLE_URL_BASE", "")):
  import sys
  # running on kaggle
  sys.path.insert(0, "/kaggle/input/hsm-source-files")
else:
  # running locally
  sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..")))

import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

import torch
import joblib

from src.utils.utils import get_raw_data_dir, get_processed_data_dir, get_submission_csv_path, set_seeds, get_models_save_path, running_in_kaggle, fill_nan_with_mean
from src.utils.constants import Constants
from src.datasets.eeg_processor import EEGDataProcessor
from src.utils.k_folds_creator import KFoldCreator

from tqdm import tqdm

set_seeds(42)

warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

2025-10-10 14:56:47,328 :: root :: INFO :: Initialising Utils
2025-10-10 14:56:47,330 :: root :: INFO :: Initialising Datasets


In [3]:
# never train on kaggle, only evaluate
SKIP_TRAIN = SKIP_TRAIN or running_in_kaggle()
print(f"SKIP_TRAIN = {SKIP_TRAIN}")

In [ ]:
DATA_PATH = get_raw_data_dir()

processor = EEGDataProcessor(raw_data_path=DATA_PATH, processed_data_path=get_processed_data_dir())
train_df = processor.process_data(vote_method=DATA_PREPARATION_VOTE_METHOD, skip_parquet=True)

test_df = pd.read_csv(DATA_PATH / "test.csv")

kl_score = nn.KLDivLoss(reduction="batchmean")


models_save_path = get_models_save_path(EXISTING_CHECKPOINT_KAGGLE_DATASET_ID) / "svm" / "spectrogram_hand_crafted" / DATA_PREPARATION_VOTE_METHOD
models_save_path.mkdir(parents=True, exist_ok=True)
print("Using models save path:", models_save_path)

Processor initialized.
Raw data path: '/home/david/git/aicomp/data'
Processed data path: '/home/david/git/aicomp/data/processed'
Starting EEG Data Processing Pipeline
Skipping NumPy file creation as requested.
Using 'max_vote_window' vote aggregation strategy.

Processed train data saved to '/home/david/git/aicomp/data/processed/train_processed.csv'.
Shape of the final dataframe: (17089, 12)

Pipeline finished successfully!
Using models save path: /home/david/git/aicomp/models/svm/spectrogram_hand_crafted/max_vote_window


## Load Spectrogram Files into Memory

In [5]:
def get_spectrogram_content(spectrogram_file: pathlib.Path):
    spectrogram_id = int(spectrogram_file.stem.split("_")[-1])
    df = pd.read_parquet(spectrogram_file)
    columns = [col for col in df.columns if col != "time"]
    content = df[columns].values
    return spectrogram_id, content, columns

In [6]:
spectrograms_dir = DATA_PATH / "train_spectrograms"
spectrogram_files = list(spectrograms_dir.glob("*.parquet"))

if not SKIP_TRAIN:
  print(f"Found {len(spectrogram_files)} train spectrogram files to load into memory")

  spectrograms = {}
  for file in tqdm(spectrogram_files):
    spectrogram_id, content, _ = get_spectrogram_content(file)
    spectrograms[spectrogram_id] = content

  gc.collect()
  print("Loaded all train spectrograms into memory")

Found 11138 train spectrogram files to load into memory


100%|██████████| 11138/11138 [06:20<00:00, 29.30it/s]

Loaded all train spectrograms into memory


## Feature Engineering

Manually extract and engineer the following clinically relevant features from the spectrogram data:
1. Regional Band Powers: Average power in each frequency band (delta, theta, alpha, beta) for each brain region (LL, LP, RP, RL), plus total power. Regional variations help seperate generalized and lateralized events. High delta power distinguishes rythmic delta from other patterns.
2. Power Rations: Ratios of power between frequency bands (delta-theta, delta-alpha, theta-alpha, low-high). Ratios are often more discriminative than absolute powers because they're less affected by individual differences in skull thickness, electrode impedance, etc.
3. Temporal Variability: Standard deviation of each band power over time. High variability can indicate seizure activity. Rythmic patterns often have lower variability.
4. Left-right band power differences: Differences in band powers between left and right hemispheres. Lateralized events often show asymmetries.
5. Left-right power ratio: Ratio of total power between left and right hemispheres. Again, lateralized events often show asymmetries.

In total, this yields 57 features.

In [7]:
# load one spectrogram to get column structure and frequencies
sample_file = spectrogram_files[0]
_, _, all_columns = get_spectrogram_content(sample_file)

# extract frequency information from column names
LL_cols = [col for col in all_columns if col.startswith('LL_')]
LP_cols = [col for col in all_columns if col.startswith('LP_')]
RP_cols = [col for col in all_columns if col.startswith('RP_')]
RL_cols = [col for col in all_columns if col.startswith('RL_')]

# extract frequencies from column names (they're the same for all regions)
freq_array = np.array([float(col.split('_')[1]) for col in LL_cols])

# get column indices for each region
LL_idx = [all_columns.index(col) for col in LL_cols]
LP_idx = [all_columns.index(col) for col in LP_cols]
RP_idx = [all_columns.index(col) for col in RP_cols]
RL_idx = [all_columns.index(col) for col in RL_cols]

In [8]:
FEATURES = []

for region in ["LL", "LP", "RP", "RL"]:
    # band powers
    FEATURES.append(f"{region}_delta_power")
    FEATURES.append(f"{region}_theta_power")
    FEATURES.append(f"{region}_alpha_power")
    FEATURES.append(f"{region}_beta_power")
    FEATURES.append(f"{region}_total_power")

    # power ratios
    FEATURES.append(f"{region}_delta_theta_ratio")
    FEATURES.append(f"{region}_delta_alpha_ratio")
    FEATURES.append(f"{region}_theta_alpha_ratio")
    FEATURES.append(f"{region}_low_high_ratio")

    # temporal Variability
    FEATURES.append(f"{region}_delta_std")
    FEATURES.append(f"{region}_theta_std")
    FEATURES.append(f"{region}_alpha_std")
    FEATURES.append(f"{region}_beta_std")

# cross-hemispheric Features

# absolute differences
FEATURES.append("left_right_delta_diff")
FEATURES.append("left_right_theta_diff")
FEATURES.append("left_right_alpha_diff")
FEATURES.append("left_right_beta_diff")

# ratios
FEATURES.append("left_right_power_ratio")

print(f"Total number of features: {len(FEATURES)}")
print(FEATURES)

Total number of features: 57
['LL_delta_power', 'LL_theta_power', 'LL_alpha_power', 'LL_beta_power', 'LL_total_power', 'LL_delta_theta_ratio', 'LL_delta_alpha_ratio', 'LL_theta_alpha_ratio', 'LL_low_high_ratio', 'LL_delta_std', 'LL_theta_std', 'LL_alpha_std', 'LL_beta_std', 'LP_delta_power', 'LP_theta_power', 'LP_alpha_power', 'LP_beta_power', 'LP_total_power', 'LP_delta_theta_ratio', 'LP_delta_alpha_ratio', 'LP_theta_alpha_ratio', 'LP_low_high_ratio', 'LP_delta_std', 'LP_theta_std', 'LP_alpha_std', 'LP_beta_std', 'RP_delta_power', 'RP_theta_power', 'RP_alpha_power', 'RP_beta_power', 'RP_total_power', 'RP_delta_theta_ratio', 'RP_delta_alpha_ratio', 'RP_theta_alpha_ratio', 'RP_low_high_ratio', 'RP_delta_std', 'RP_theta_std', 'RP_alpha_std', 'RP_beta_std', 'RL_delta_power', 'RL_theta_power', 'RL_alpha_power', 'RL_beta_power', 'RL_total_power', 'RL_delta_theta_ratio', 'RL_delta_alpha_ratio', 'RL_theta_alpha_ratio', 'RL_low_high_ratio', 'RL_delta_std', 'RL_theta_std', 'RL_alpha_std', 'RL_b

In [9]:
def extract_clinical_features(ten_minute_window):
    features = {}
    
    # separate regions
    LL_data = ten_minute_window[:, LL_idx]
    LP_data = ten_minute_window[:, LP_idx]
    RP_data = ten_minute_window[:, RP_idx]
    RL_data = ten_minute_window[:, RL_idx]
    
    # define bands
    delta_idx = np.where((freq_array >= 0.5) & (freq_array <= 4.0))[0]
    theta_idx = np.where((freq_array >= 4.0) & (freq_array < 8.0))[0]
    alpha_idx = np.where((freq_array >= 8.0) & (freq_array < 13.0))[0]
    beta_idx = np.where((freq_array >= 13.0) & (freq_array < 30.0))[0]
    
    # band powers
    for region_name, region_data in [('LL', LL_data), ('LP', LP_data), ('RP', RP_data), ('RL', RL_data)]:
        features[f'{region_name}_delta_power'] = region_data[:, delta_idx].mean()
        features[f'{region_name}_theta_power'] = region_data[:, theta_idx].mean()
        features[f'{region_name}_alpha_power'] = region_data[:, alpha_idx].mean()
        features[f'{region_name}_beta_power'] = region_data[:, beta_idx].mean()
        features[f'{region_name}_total_power'] = region_data.mean()
    
    # power ratios
    for region_name, region_data in [('LL', LL_data), ('LP', LP_data), ('RP', RP_data), ('RL', RL_data)]:
        features[f'{region_name}_delta_theta_ratio'] = features[f'{region_name}_delta_power'] / (features[f'{region_name}_theta_power'] + 1e-10)
        features[f'{region_name}_delta_alpha_ratio'] = features[f'{region_name}_delta_power'] / (features[f'{region_name}_alpha_power'] + 1e-10)
        features[f'{region_name}_theta_alpha_ratio'] = features[f'{region_name}_theta_power'] / (features[f'{region_name}_alpha_power'] + 1e-10)
        features[f'{region_name}_low_high_ratio'] = (features[f'{region_name}_delta_power'] + features[f'{region_name}_theta_power']) / (features[f'{region_name}_alpha_power'] + features[f'{region_name}_beta_power'] + 1e-10)
    
    # temporal Variability
    for region_name, region_data in [('LL', LL_data), ('LP', LP_data), ('RP', RP_data), ('RL', RL_data)]:
        delta_power_over_time = region_data[:, delta_idx].mean(axis=1)
        theta_power_over_time = region_data[:, theta_idx].mean(axis=1)
        alpha_power_over_time = region_data[:, alpha_idx].mean(axis=1)
        beta_power_over_time = region_data[:, beta_idx].mean(axis=1)

        features[f'{region_name}_delta_std'] = delta_power_over_time.std()
        features[f'{region_name}_theta_std'] = theta_power_over_time.std()
        features[f'{region_name}_alpha_std'] = alpha_power_over_time.std()
        features[f'{region_name}_beta_std'] = beta_power_over_time.std()

    # cross-hemispheric Features

    # absolute differences
    features['left_right_delta_diff'] = abs(
        (features['LL_delta_power'] + features['LP_delta_power']) - 
        (features['RL_delta_power'] + features['RP_delta_power'])
    )

    features['left_right_theta_diff'] = abs(
        (features['LL_theta_power'] + features['LP_theta_power']) - 
        (features['RL_theta_power'] + features['RP_theta_power'])
    )

    features['left_right_alpha_diff'] = abs(
        (features['LL_alpha_power'] + features['LP_alpha_power']) - 
        (features['RL_alpha_power'] + features['RP_alpha_power'])
    )

    features['left_right_beta_diff'] = abs(
        (features['LL_beta_power'] + features['LP_beta_power']) - 
        (features['RL_beta_power'] + features['RP_beta_power'])
    )

    # total power ratio
    left_total = features['LL_total_power'] + features['LP_total_power']
    right_total = features['RL_total_power'] + features['RP_total_power']
    features['left_right_power_ratio'] = left_total / (right_total + 1e-10)
    
    return features

In [10]:
def extract_spectrogram_features(ten_minute_window):
  features_dict = extract_clinical_features(ten_minute_window)
  return np.array([features_dict[feat_name] for feat_name in FEATURES]) # return features in the same order as FEATURES list

In [11]:
if not SKIP_TRAIN:
  data = np.zeros((len(train_df), len(FEATURES)))

  def extract_train_spectrogram_features(row, all_spectrograms):
    spectrogram_id = int(row["spectrogram_id"])
    middle_offset = (row["min_offset"] + row["max_offset"]) // 2 # this the middle between the least spectrogram offset and greatest spectogram offset
    row_index = int(middle_offset // 2) # each spectrogram row corresponds to 2s, so we divide by 2 to get the row index
    window = np.array(all_spectrograms[spectrogram_id][row_index:row_index+300,:])
    average_frequencies = extract_spectrogram_features(window)
    return average_frequencies

  for i in tqdm(range(len(train_df)), total=len(train_df)):
    row = train_df.iloc[i]
    average_features = extract_train_spectrogram_features(row, spectrograms)
    data[i,:] = average_features

100%|██████████| 17089/17089 [00:18<00:00, 938.21it/s]


In [12]:
if not SKIP_TRAIN:
  train_df[FEATURES] = data

  print(train_df.head())

   eeg_id  spectrogram_id  patient_id expert_consensus  seizure_vote  \
0  568657       789577333       20654            Other           0.0   
1  582999      1552638400       20230              LPD           0.0   
2  642382        14960202        5955            Other           0.0   
3  751790       618728447       38549              GPD           0.0   
4  778705        52296320       40955            Other           0.0   

   lpd_vote  gpd_vote  lrda_vote  grda_vote  other_vote  ...  \
0  0.000000      0.25   0.000000   0.166667    0.583333  ...   
1  0.857143      0.00   0.071429   0.000000    0.071429  ...   
2  0.000000      0.00   0.000000   0.000000    1.000000  ...   
3  0.000000      1.00   0.000000   0.000000    0.000000  ...   
4  0.000000      0.00   0.000000   0.000000    1.000000  ...   

   RL_low_high_ratio  RL_delta_std  RL_theta_std  RL_alpha_std  RL_beta_std  \
0          33.868639   2176.569336    192.093048     60.546051    23.950325   
1          23.379087    

# Train SVM Model

In [13]:
N_SPLITS = 5
targets_dict = {"Seizure":0, "LPD":1, "GPD":2, "LRDA":3, "GRDA":4, "Other":5}

In [14]:
if not SKIP_TRAIN:
    fold_creator = KFoldCreator(n_splits=N_SPLITS, seed=Constants.SEED)
    train_folds_df = fold_creator.create_folds(
        df=train_df, stratify_col="expert_consensus", group_col="patient_id"
    )

In [16]:
if not SKIP_TRAIN:
    all_oof = []
    all_true = []

    for fold in range(N_SPLITS):
        fold_train_df = train_folds_df[train_folds_df["fold"] != fold].reset_index(drop=True)
        fold_valid_df = train_folds_df[train_folds_df["fold"] == fold].reset_index(drop=True)

        print("=" * 40)
        print(f"FOLD {fold}")
        print(f"Train size: {len(fold_train_df)}, Valid size: {len(fold_valid_df)}")
        print("=" * 30)

        X_train = fold_train_df[FEATURES].values
        y_train = fold_train_df["expert_consensus"].map(targets_dict).values

        X_valid = fold_valid_df[FEATURES].values
        y_valid = fold_valid_df["expert_consensus"].map(targets_dict).values

        X_train = fill_nan_with_mean(X_train)
        X_valid = fill_nan_with_mean(X_valid)

        # scale features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_valid_scaled = scaler.transform(X_valid)

        # train SVM with probability estimates
        model = SVC(
            kernel='rbf',
            C=1.0,
            gamma='scale',
            probability=True,
            random_state=Constants.SEED,
            verbose=False,
            cache_size=1000 # 1 GB
        )
        
        print("Training SVM...")
        model.fit(X_train_scaled, y_train)

        joblib.dump(model, models_save_path / f"fold_{fold}_model.pkl")
        joblib.dump(scaler, models_save_path / f"fold_{fold}_scaler.pkl")
        
        print("Predicting on validation set...")
        oof = model.predict_proba(X_valid_scaled)
        all_oof.extend(oof)

        all_true.extend(fold_valid_df[Constants.TARGETS].values)

        del X_train, y_train, X_valid, y_valid, X_train_scaled, X_valid_scaled, oof
        gc.collect()

    all_oof = np.array(all_oof)
    all_true = np.array(all_true)

FOLD 0
Train size: 13022, Valid size: 4067
Training SVM...
Predicting on validation set...
FOLD 1
Train size: 13431, Valid size: 3658
Training SVM...
Predicting on validation set...
FOLD 2
Train size: 13708, Valid size: 3381
Training SVM...
Predicting on validation set...
FOLD 3
Train size: 14464, Valid size: 2625
Training SVM...
Predicting on validation set...
FOLD 4
Train size: 13731, Valid size: 3358
Training SVM...
Predicting on validation set...


## CV Score

In [24]:
if not SKIP_TRAIN:
  all_oof_tensor = torch.tensor(all_oof, dtype=torch.float32)
  all_true_tensor = torch.tensor(all_true, dtype=torch.float32)

  kl_score = nn.KLDivLoss(reduction="batchmean")
  score = kl_score(all_oof_tensor.log(), all_true_tensor).item()

  print(f"OOF KL Score: {score}")

OOF KL Score: 1.1904058456420898


## Infer on Test and create Submission

In [25]:
test_spectrograms_dir = DATA_PATH / "test_spectrograms"
test_spectrogram_files = list(test_spectrograms_dir.glob("*.parquet"))
print(f"Found {len(test_spectrogram_files)} test spectrogram files to load into memory")

test_spectrograms = {}
for file in tqdm(test_spectrogram_files):
  spectrogram_id, content, _ = get_spectrogram_content(file)
  test_spectrograms[spectrogram_id] = content

gc.collect()
print("Loaded all test spectrograms into memory")

Found 1 test spectrogram files to load into memory


100%|██████████| 1/1 [00:00<00:00, 28.15it/s]

Loaded all test spectrograms into memory


In [26]:
test_data = np.zeros((len(test_df), len(FEATURES)))

def extract_test_spectrogram_features(row, all_spectrograms):
  # this differs from train because all test spectrograms are exactly 10 minutes long, so we don't need to extract the center window
  spectrogram_id = int(row["spectrogram_id"])
  content = np.array(all_spectrograms[spectrogram_id][:])
  average_frequencies = extract_spectrogram_features(content)
  return average_frequencies

for i in tqdm(range(len(test_df)), total=len(test_df)):
  row = test_df.iloc[i]
  average_features = extract_test_spectrogram_features(row, test_spectrograms)
  test_data[i,:] = average_features

100%|██████████| 1/1 [00:00<00:00, 543.94it/s]


In [27]:
test_df[FEATURES] = test_data
test_df.head()

,spectrogram_id,eeg_id,patient_id,LL_delta_power,LL_theta_power,LL_alpha_power,LL_beta_power,LL_total_power,LL_delta_theta_ratio,LL_delta_alpha_ratio,...,RL_low_high_ratio,RL_delta_std,RL_theta_std,RL_alpha_std,RL_beta_std,left_right_delta_diff,left_right_theta_diff,left_right_alpha_diff,left_right_beta_diff,left_right_power_ratio
0,853520,3911565283,6885,5.342817,0.455045,0.621987,1.349531,1.700264,11.741293,8.589915,...,79.804853,32.726185,0.364277,0.092094,0.072847,23.825184,0.79198,0.379858,1.352977,0.536792


In [28]:
test_preds = []

for fold in range(N_SPLITS):
  print("=" * 40)
  print(f"Predicting fold {fold}")
  print("=" * 40)

  X_test = test_df[FEATURES].values
  X_test = fill_nan_with_mean(X_test)

  scaler = joblib.load(models_save_path / f"fold_{fold}_scaler.pkl")
  model = joblib.load(models_save_path / f"fold_{fold}_model.pkl")

  X_test_scaled = scaler.transform(X_test)

  preds = model.predict_proba(X_test_scaled)
  test_preds.append(preds)

test_preds = np.mean(test_preds, axis=0)
print(f"Test predictions shape: {test_preds.shape}")

Predicting fold 0
Predicting fold 1
Predicting fold 2
Predicting fold 3
Predicting fold 4
Test predictions shape: (1, 6)


In [29]:
# sanity check: all predictions should sum to 1
assert np.allclose(test_preds.sum(axis=1), 1.0)

In [30]:
submission = pd.DataFrame({"eeg_id": test_df["eeg_id"]})
submission[Constants.TARGETS] = test_preds

submission.to_csv(get_submission_csv_path(), index=False)